## Keras implementation of https://junyanz.github.io/CycleGAN/

In [ ]:
import os
os.environ['KERAS_BACKEND']='tensorflow' # can choose theano, tensorflow, cntk
os.environ['THEANO_FLAGS']='floatX=float32,device=cuda,optimizer=fast_run,dnn.library_path=/usr/lib'
#os.environ['THEANO_FLAGS']='floatX=float32,device=cuda,optimizer=fast_compile,dnn.library_path=/usr/lib'

In [ ]:
import keras.backend as K
if os.environ['KERAS_BACKEND'] =='theano':
    channel_axis=1
    K.set_image_data_format('channels_first')
    channel_first = True
else:
    K.set_image_data_format('channels_last')
    channel_axis=-1
    channel_first = False

In [ ]:
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, Dropout
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten
from keras.layers import Concatenate, ZeroPadding2D, Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal

In [ ]:
# Weights initializations
# bias are initailized as 0
def __conv_init(a):
    print("conv_init", a)
    k = RandomNormal(0, 0.02)(a) # for convolution kernel
    k.conv_weight = True    
    return k
conv_init = RandomNormal(0, 0.02)
gamma_init = RandomNormal(1., 0.02) # for batch normalization


In [ ]:
# HACK speed up theano
if K._BACKEND == 'theano':
    import keras.backend.theano_backend as theano_backend
    def _preprocess_conv2d_kernel(kernel, data_format):
        #return kernel
        if hasattr(kernel, "original"):
            print("use original")
            return kernel.original
        elif hasattr(kernel, '_keras_shape'):
            s = kernel._keras_shape
            print("use reshape",s)
            kernel = kernel.reshape((s[3], s[2],s[0], s[1]))
        else:
            kernel = kernel.dimshuffle((3, 2, 0, 1))
        return kernel
    theano_backend._preprocess_conv2d_kernel = _preprocess_conv2d_kernel

In [ ]:
# Basic discriminator
def conv2d(f, *a, **k):
    return Conv2D(f, kernel_initializer = conv_init, *a, **k)
def batchnorm():
    return BatchNormalization(momentum=0.9, axis=channel_axis, epsilon=1.01e-5,
                                   gamma_initializer = gamma_init)
def BASIC_D(nc_in, ndf, input_size = (None, None), max_layers=3, use_sigmoid=True):
    """DCGAN_D(nc, ndf, max_layers=3)
       nc: channels
       ndf: filters of the first layer
       max_layers: max hidden layers
    """    
    if channel_first:
        input_a =  Input(shape=(nc_in, *input_size))
    else:
        input_a = Input(shape=(*input_size, nc_in))
    _ = input_a
    _ = conv2d(ndf, kernel_size=4, strides=2, padding="same", name = 'First') (_)
    _ = LeakyReLU(alpha=0.2)(_)
    
    for layer in range(1, max_layers):        
        out_feat = ndf * min(2**layer, 8)
        _ = conv2d(out_feat, kernel_size=4, strides=2, padding="same", 
                   use_bias=False, name = 'pyramid.{0}'.format(layer)             
                        ) (_)
        _ = batchnorm()(_, training=1)        
        _ = LeakyReLU(alpha=0.2)(_)
    
    out_feat = ndf*min(2**max_layers, 8)
    _ = ZeroPadding2D(1)(_)
    _ = conv2d(out_feat, kernel_size=4,  use_bias=False, name = 'pyramid_last') (_)
    _ = batchnorm()(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    
    # final layer
    _ = ZeroPadding2D(1)(_)
    _ = conv2d(1, kernel_size=4, name = 'final'.format(out_feat, 1)) (_)    
    
    _ = Flatten(name = 'flatten') (_)
    _ = Dense(1, name = 'final_dense', activation = "sigmoid" if use_sigmoid else None) (_)
    
    return Model(inputs=[input_a], outputs=_)

In [ ]:
def UNET_G(isize, nc_in=3, nc_out=3, ngf=64, fixed_input_size=True):    
    max_nf = 8*ngf    
    def block(x, s, nf_in, use_batchnorm=True, nf_out=None, nf_next=None):
        # print("block",x,s,nf_in, use_batchnorm, nf_out, nf_next)
        assert s>=2 and s%2==0
        if nf_next is None:
            nf_next = min(nf_in*2, max_nf)
        if nf_out is None:
            nf_out = nf_in
        x = conv2d(nf_next, kernel_size=4, strides=2, use_bias=(not (use_batchnorm and s>2)),
                   padding="same", name = 'conv_{0}'.format(s)) (x)
        if s>2:
            if use_batchnorm:
                x = batchnorm()(x, training=1)
            x2 = LeakyReLU(alpha=0.2)(x)
            x2 = block(x2, s//2, nf_next)
            x = Concatenate(axis=channel_axis)([x, x2])            
        x = Activation("relu")(x)
        x = Conv2DTranspose(nf_out, kernel_size=4, strides=2,
                            use_bias=not use_batchnorm,
                            kernel_initializer = conv_init,          
                            name = 'convt.{0}'.format(s))(x)        
        x = Cropping2D(1)(x)
        if use_batchnorm:
            x = batchnorm()(x, training=1)
        if s <=8:
            x = Dropout(0.5)(x, training=1)
        return x
    
    s = isize if fixed_input_size else None
    if channel_first:
        _ = inputs = Input(shape=(nc_in, s, s))
    else:
        _ = inputs = Input(shape=(s, s, nc_in))        
    _ = block(_, isize, nc_in, False, nf_out=nc_out, nf_next=ngf)
    _ = Activation('tanh')(_)
    return Model(inputs=inputs, outputs=[_])

In [ ]:
def UNET_G_SM(isize, nc_in=3, nc_out=3, ngf=64, fixed_input_size=True):    
    max_nf = 8*ngf    
    def block(x, s, nf_in, use_batchnorm=True, nf_out=None, nf_next=None):
        # print("block",x,s,nf_in, use_batchnorm, nf_out, nf_next)
        assert s>=2 and s%2==0
        if nf_next is None:
            nf_next = min(nf_in*2, max_nf)
        if nf_out is None:
            nf_out = nf_in
        x = conv2d(nf_next, kernel_size=4, strides=2, use_bias=(not (use_batchnorm and s>2)),
                   padding="same", name = 'conv_{0}'.format(s)) (x)
        if s>2:
            if use_batchnorm:
                x = batchnorm()(x, training=1)
            x2 = LeakyReLU(alpha=0.2)(x)
            x2 = block(x2, s//2, nf_next)
            x = Concatenate(axis=channel_axis)([x, x2])            
        x = Activation("relu")(x)
        x = Conv2DTranspose(nf_out, kernel_size=4, strides=2,
                            use_bias=not use_batchnorm,
                            kernel_initializer = conv_init,          
                            name = 'convt.{0}'.format(s))(x)        
        x = Cropping2D(1)(x)
        if use_batchnorm:
            x = batchnorm()(x, training=1)
        if s <=8:
            x = Dropout(0.5)(x, training=1)
        return x
    
    s = isize if fixed_input_size else None
    if channel_first:
        _ = inputs = Input(shape=(nc_in, s, s))
    else:
        _ = inputs = Input(shape=(s, s, nc_in))        
    _ = block(_, isize, nc_in, False, nf_out=ngf, nf_next=ngf)
    _ = Activation('relu')(_)
    _ = Conv2DTranspose(ngf, kernel_size=(13, 4), strides=(1, 2),
                            use_bias=True,
                            kernel_initializer = conv_init,
                            name = 'convt.fin')(_)
    _ = Cropping2D((0, 1))(_)
    
    _ = Activation("relu")(_)
    _ = Conv2DTranspose(ngf//2, kernel_size=4, strides=2,
                        use_bias=True,
                        kernel_initializer = conv_init,          
                        name = 'convt.finfin')(_)        
    _ = Cropping2D(1)(_)
    _ = Activation('relu')(_)
    _ = Conv2DTranspose(nc_out, kernel_size=(3, 4), strides=(1, 2),
                        use_bias=True,
                        kernel_initializer = conv_init,          
                        name = 'convt.finfinfin')(_)        
    _ = Cropping2D(1)(_)
    _ = Activation('tanh')(_)
    return Model(inputs=inputs, outputs=[_])

In [ ]:
def UNET_G_MS(isize, nc_in=1, nc_out=3, ngf=64, fixed_input_size=True):    
    max_nf = 8*ngf    
    def block(x, s, nf_in, use_batchnorm=True, nf_out=None, nf_next=None):
        # print("block",x,s,nf_in, use_batchnorm, nf_out, nf_next)
        assert s>=2 and s%2==0
        if nf_next is None:
            nf_next = min(nf_in*2, max_nf)
        if nf_out is None:
            nf_out = nf_in
        x = conv2d(nf_next, kernel_size=4, strides=2, use_bias=(not (use_batchnorm and s>2)),
                   padding="same", name = 'conv_{0}'.format(s)) (x)
        if s>2:
            if use_batchnorm:
                x = batchnorm()(x, training=1)
            x2 = LeakyReLU(alpha=0.2)(x)
            x2 = block(x2, s//2, nf_next)
            x = Concatenate(axis=channel_axis)([x, x2])            
        x = Activation("relu")(x)
        x = Conv2DTranspose(nf_out, kernel_size=4, strides=2,
                            use_bias=not use_batchnorm,
                            kernel_initializer = conv_init,
                            name = 'convt.{0}'.format(s))(x)        
        x = Cropping2D(1)(x)
        if use_batchnorm:
            x = batchnorm()(x, training=1)
        if s <=8:
            x = Dropout(0.5)(x, training=1)
        return x
    
    s = isize if fixed_input_size else None
    if channel_first:
        _ = inputs = Input(shape=(nc_in, *musicSize))
    else:
        _ = inputs = Input(shape=(*musicSize, nc_in))
    _ = conv2d(ngf, kernel_size=4, strides=(1, 2), use_bias=True,
                   padding="same", name = 'conv_superfirst') (_)
    _ = LeakyReLU(alpha=0.2)(_)
    #_ = ZeroPadding2D(padding=(0, 1), data_format=None) (_)
    _ = conv2d(ngf*2, kernel_size=4, strides=2, use_bias=True,
                   padding="same", name = 'conv_first') (_)
    _ = LeakyReLU(alpha=0.2)(_)
    _ = ZeroPadding2D(padding=(0, 1), data_format=None) (_)
    _ = conv2d(ngf*2, kernel_size=(13, 4), strides=(1,2), use_bias=True,
               padding="valid", name = 'conv_firstfirst') (_)
    _ = LeakyReLU(alpha=0.2)(_)
    _ = block(_, isize, ngf*2, False, nf_out=nc_out, nf_next=ngf)
    _ = Activation('tanh')(_)
    return Model(inputs=inputs, outputs=[_])

In [ ]:
nc_s = 3
nc_m = 1
ngf = 64
ndf = 128

spriteSize = (32, 32)
musicSize = (88, 256)
fs = 32
snap_to = 1 # snaps to every 1/4 of a second
snap_factor = 1 / snap_to
dur = 8.0
pitch_range = (21, 109)
binary_music = True

use_lsgan = True
λ = 10 if use_lsgan else 100
md_weight = 1
mg_weight = 2

continue_training = False
batchSize = 2
lrD = 2e-4
lrG = 2e-4

assert((fs // snap_factor) * snap_factor == fs)

In [ ]:
netGM = UNET_G_SM(32, nc_s, nc_m, ngf) #netGA
netGM.summary()

In [ ]:
netGS = UNET_G_MS(32, nc_m, nc_s, ngf) #netGB
netGS.summary()

In [ ]:
netDM = BASIC_D(nc_m, ndf, input_size = musicSize, use_sigmoid = not use_lsgan) # netDA
netDS = BASIC_D(nc_s, ndf, input_size = spriteSize, use_sigmoid = not use_lsgan) # netDB
netDM.summary()
netDS.summary()


In [ ]:
from keras.optimizers import RMSprop, SGD, Adam

In [ ]:
if use_lsgan:
    loss_fn = lambda output, target : K.mean(K.abs(K.square(output-target)))
else:
    loss_fn = lambda output, target : -K.mean(K.log(output+1e-12)*target+K.log(1-output+1e-12)*(1-target))

def cycle_variables(netG1, netG2):
    real_input = netG1.inputs[0]
    fake_output = netG1.outputs[0]
    rec_input = netG2([fake_output])
    fn_generate = K.function([real_input], [fake_output, rec_input])
    return real_input, fake_output, rec_input, fn_generate

real_M, fake_S, rec_M, cycleM_generate = cycle_variables(netGS, netGM)
real_S, fake_M, rec_S, cycleS_generate = cycle_variables(netGM, netGS)

In [ ]:
def D_loss(netD, real, fake, rec):
    output_real = netD([real])
    output_fake = netD([fake])
    loss_D_real = loss_fn(output_real, K.ones_like(output_real))
    loss_D_fake = loss_fn(output_fake, K.zeros_like(output_fake))
    loss_G = loss_fn(output_fake, K.ones_like(output_fake))
    loss_D = loss_D_real+loss_D_fake
    loss_cyc = K.mean(K.abs(rec-real))
    return loss_D, loss_G, loss_cyc

loss_DM, loss_GM, loss_cycS = D_loss(netDM, real_M, fake_M, rec_M)
loss_DS, loss_GS, loss_cycM = D_loss(netDS, real_S, fake_S, rec_S)
loss_cyc = loss_cycM+loss_cycS

In [ ]:
loss_G = loss_GM+loss_GS+λ*loss_cyc
loss_D = loss_DM+loss_DS

weightsD = netDM.trainable_weights + netDS.trainable_weights
weightsG = netGM.trainable_weights + netGS.trainable_weights

training_updates = Adam(lr=lrD, beta_1=0.5).get_updates(weightsD,[],loss_D)
netD_train = K.function([real_M, real_S],[md_weight * loss_DM/2, loss_DS/2], training_updates)
training_updates = Adam(lr=lrG, beta_1=0.5).get_updates(weightsG,[], loss_G)
netG_train = K.function([real_M, real_S], [mg_weight * loss_GM, loss_GS, loss_cyc], training_updates)

In [ ]:
!pip install pretty_midi
import pretty_midi

def prepare_midi(pm, fs = 16, duration = 30.0, pitchrange = (0, 128)):
    """ 
    Takes a given midi file name and returns a PrettyMIDI object and its piano roll. 

    Extended description of function. 

    Parameters: 
    name (str): Name of midi file
    fs (int): Samples per sec
    duration (float): duration of time to keep 
    pitchrange (tuple[int]): tuple of starting pitch and ending pitch. 0 to 127 possible

    Returns: 
    (PrettyMIDI, numpyArray): prettymidi object and cropped piano roll array

    """ 

    pmroll = pm.get_piano_roll(fs)[pitchrange[0] : pitchrange[1]]
    if duration > 0:
        pmroll = pmroll[:, 0 : int(duration * fs)]
    return pmroll

def to_midi(pr, fs = 8, pitchrange = (0, 128), force_velocity = False):
    pm = pretty_midi.PrettyMIDI()
    piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program = piano_program)
    
    pitch_start = pitchrange[0]
    for p in range(len(pr)):
        start = False
        startTime = 0.0
        v = 0
        for t in range(len(pr[p])):
            if not start and int(pr[p][t]) > 0:
                start = True
                v = int(pr[p][t])
                startTime = t / fs
            elif start and int(pr[p][t]) != v:
                endTime = t / fs
                vel = v
                if force_velocity:
                    vel = 125
                _ = pretty_midi.Note(velocity = vel, 
                                    pitch = pitch_start + p, start = startTime, end = endTime)
                piano.notes.append(_)
                if int(pr[p][t]) == 0:
                    start = False
                else:
                    startTime = endTime
                    v = int(pr[p][t])
    
    pm.instruments.append(piano)
    return pm


In [ ]:
def show_pitch_occur(midis):
    pitches = [0] * 128
    for fn in midis:        
        try:
            _ = pretty_midi.PrettyMIDI(fn)
            print(fn, "\n loading... loaded")
        except:
            print(fn, "\n loading... ERROR")
        roll = prepare_midi(_, fs = 8, duration = 0.0)
        for p in range(128):
            if len(set(roll[p])) > 1:
                pitches[p] += 1

    for p in range(128):
        print(p, pitches[p], sep=': ')

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import glob
from random import randint, shuffle

def load_data(file_pattern):
    return glob.glob(file_pattern, recursive = True)

def read_image(fn, imgSize, augment=False):
    im = Image.open(fn).convert("RGBA")
    im = im.resize(imgSize, Image.BILINEAR )
    img = np.array(im, dtype=np.float64)
    alphaLayer = img[:,:,3] / 255.0
    img[:,:,0] *= alphaLayer
    img[:,:,1] *= alphaLayer
    img[:,:,2] *= alphaLayer
    img = (img/255.0*2-1)[:,:,:3]
    if augment and randint(0,1):
        img=img[:,::-1]
    if channel_first:        
        img = np.moveaxis(img, 2, 0)
    return img

def read_midi(fn, imgSize, fs, duration, prange, augment=False):
    try:
        _ = pretty_midi.PrettyMIDI(fn)
    except:
        return None
    roll = prepare_midi(_, fs, 0, prange)
    window = int(fs * duration)
    end = len(roll[0]) - int(5 * fs) - window
    if end < int(5 * fs):
        print(end, " not big enough (", fn, ")")
        return None
    start = np.random.randint(int(5 * fs), end)
    snap = fs // snap_factor
    start = int((start // snap) * snap)
    img = roll[:, start : start + window]
    if binary_music:
        img = np.clip(img, 0, 1)
    else:
        img = (img/128.0)
    if augment and randint(0,1):
        img=img[:,::-1]
    if channel_first:        
        img = np.moveaxis(img, 2, 0)
    if img.shape != imgSize:
        print("[ERROR] Created an invalid shaped piece of music data! Size: ", img.shape, " instead of ", imgSize)
    return img

data = "sprites2music"
train_S = load_data('trainingSet/sprites/**/*.png')
train_M = load_data('trainingSet/music/**/*.mid')

print("[Removing songs that are too short]")
to_remove = []
for m in train_M:
    if read_midi(m, musicSize, fs, dur, pitch_range) is None:
        to_remove.append(m)
for m in to_remove:
    train_M.remove(m)

assert len(train_M) and len(train_S)

shuffle(train_M)
shuffle(train_S)

print("Num midi files: ", len(train_M))
print("Num sprites: ", len(train_S))

In [ ]:
#show_pitch_occur(train_M)
test = None
while test is None:
    n = np.random.randint(len(train_M))
    print(n, train_M[n])
    test = read_midi(train_M[n], musicSize, fs, dur, pitch_range)
plt.figure(figsize = (40, 80))
plt.imshow(test)
plt.show()

pm = to_midi(test, fs, pitch_range, force_velocity= binary_music)
pm.write('test.mid')
print(test.shape)

In [ ]:
def minibatchS(data, batchsize, imgSize = spriteSize, augment = False):
    length = len(data)
    epoch = i = 0
    tmpsize = None    
    while True:
        size = tmpsize if tmpsize else batchsize
        if i+size > length:
            shuffle(data)
            i = 0
            epoch+=1        
        rtn = [read_image(data[j], imgSize, augment) for j in range(i,i+size)]
        i+=size
        tmpsize = yield epoch, np.float32(rtn)       
        
def minibatchM(data, batchsize, imgSize = musicSize, augment = False):
    length = len(data)
    epoch = i = 0
    tmpsize = None    
    while True:
        size = tmpsize if tmpsize else batchsize
        if i+size > length:
            shuffle(data)
            i = 0
            epoch+=1        
        rtn = [read_midi(data[j], imgSize, fs, dur, pitch_range, augment) for j in range(i,i+size)]
        i+=size
        tmpsize = yield epoch, np.expand_dims(np.float32(rtn), -1)

def minibatchMS(dataM, dataS, batchsize):
    batchM=minibatchM(dataM, batchsize, augment=False)
    batchS=minibatchS(dataS, batchsize, augment=True)
    tmpsize = None
    while True:        
        ep1, M = batchM.send(tmpsize)
        ep2, S = batchS.send(tmpsize)
        tmpsize = yield max(ep1, ep2), M, S

In [ ]:
def showX(X, imageWidth, imageHeight, nc, rows=1, figsize = (20, 40)):
    assert X.shape[0]%rows == 0
    cols = X.shape[0]//rows
    plt.figure(figsize=figsize)
    for r in range(rows):
        for c in range(cols):
            i = r*cols + c
            int_X = X[i]
            if (imageHeight, imageWidth) == spriteSize:
                int_X = ((int_X+1)/2)
            int_X = (int_X*255).clip(0,255).astype('uint8')
            if channel_first:
                int_X = np.moveaxis(int_X.reshape(-1,nc,imageWidth, imageHeight), 1, nc)
            else:
                int_X = int_X.reshape(-1, imageWidth, imageHeight, nc)
            ax = plt.subplot(rows, cols, i+1)
            ax.imshow(np.squeeze(int_X), cmap=plt.get_cmap('gray'), vmin=0, vmax=255)
            ax.axis('off')
    plt.show()

In [ ]:
train_batch = minibatchMS(train_M, train_S, 4)

_, M, S = next(train_batch)
showX(M, *musicSize, nc_m)
showX(S, *spriteSize, nc_s)
del train_batch, M, S

In [ ]:
def showG(M,S):
    def G(fn_generate, X):
        fakes = []
        recs = []
        for i in range(X.shape[0]):
            fake, rec = fn_generate([X[i:i+1]])
            fakes.append(fake[0])
            recs.append(rec[0])
        return [np.array(fakes), np.array(recs)]
    rM = G(cycleM_generate, M)
    rS = G(cycleS_generate, S)
    arr = [M,*rM,S,*rS]
    for elem in arr:
        print(elem.shape)
        showX(elem, elem.shape[1], elem.shape[2], elem.shape[3], 1)

In [ ]:
import time
from IPython.display import clear_output
t0 = time.time()
niter = 200
gen_iterations = 0
epoch = 0
errCyc_sum = errGM_sum = errGS_sum = errDM_sum = errDS_sum = 0

past_epoch = 0
display_iters = 25
#val_batch = minibatch(valAB, 6, direction)
train_batch = minibatchMS(train_M, train_S, batchSize)

while epoch < niter: 
    epoch, M, S = next(train_batch)  
    errDM, errDS  = netD_train([M, S])
    errDM_sum +=errDM
    errDS_sum +=errDS

    # epoch, trainA, trainB = next(train_batch)
    errGM, errGS, errCyc = netG_train([M, S])
    errGM_sum += errGM
    errGS_sum += errGS
    errCyc_sum += errCyc
    gen_iterations+=1
    if gen_iterations%display_iters==0:
        #if gen_iterations%(5*display_iters)==0:
        #clear_output()
        print('[%d/%d][%d] Loss_D: %f %f Loss_G: %f %f loss_cyc %f'
        % (epoch, niter, gen_iterations, errDM_sum/display_iters, errDS_sum/display_iters,
           errGM_sum/display_iters, errGS_sum/display_iters, 
           errCyc_sum/display_iters), "%.3f" % ((time.time()-t0)/60.0))
        errCyc_sum = errGM_sum = errGS_sum = errDM_sum = errDS_sum = 0
    if epoch != past_epoch:
        _, M, S = train_batch.send(4)
        showG(M,S)
        past_epoch = epoch

In [ ]:
# save all the things
netDM.save('run06_netDM')
netDS.save('run06_netDS')
netGM.save('run06_netGM')
netGS.save('run06_netGS')